In [37]:
import pandas as pd
import statsmodels.api as sm

from scipy.stats import norm

import numpy as np



### Problem (a)

In [40]:

data = pd.read_csv('TitanicData.csv')

data = data.dropna(subset=['Survived', 'Pclass', 'Sex', 'Age'])


data['Sex'] = data['Sex'].apply(lambda x: 1 if x == 'male' else 0)

X = data[['Pclass', 'Sex', 'Age']]
X = sm.add_constant(X)  # Adds the intercept term
y = data['Survived']  # Dependent variable

probit_model = sm.Probit(y, X)
probit_results = probit_model.fit()

print(probit_results.summary())

Optimization terminated successfully.
         Current function value: 0.455180
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                         Probit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Wed, 20 Nov 2024   Pseudo R-squ.:                  0.3261
Time:                        14:13:55   Log-Likelihood:                -325.00
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.167e-68
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.8693      0.270     10.646      0.000       2.341       3.398
Pclass        -0.7206      0.

### Problem (b)

In [43]:
data_1 = pd.DataFrame({'const': 1, 'Pclass': [3], 'Sex': [1], 'Age': [55]})
survival_probability_1 = probit_results.predict(data_1)[0]
survival_probability_1

0.029196061705704688

### Problem (c)

In [46]:
data_2 = pd.DataFrame({'const': 1, 'Pclass': [1], 'Sex': [1], 'Age': [55]})
survival_probability_2 = probit_results.predict(data_2)[0]
survival_probability_2

0.32581295553857403

In [48]:
survival_probability_2 - survival_probability_1

0.2966168938328693

### Problem (d)

In [51]:
cov_matrix

array([[ 7.26473604e-02, -1.75270537e-02, -9.53719737e-03,
        -8.46824303e-04],
       [-1.75270537e-02,  5.74983534e-03,  7.17604974e-04,
         1.48195670e-04],
       [-9.53719737e-03,  7.17604974e-04,  1.35959126e-02,
        -2.00388110e-05],
       [-8.46824303e-04,  1.48195670e-04, -2.00388110e-05,
         1.82517194e-05]])

In [53]:
beta = probit_results.params.values

z_class_3 = beta[0] + beta[1] * 3 + beta[2] * 1 + beta[3] * 55
z_class_1 = beta[0] + beta[1] * 1 + beta[2] * 1 + beta[3] * 55

# PDF values at z for class 3 and class 1 (φ(z))
pdf_class_3 = norm.pdf(z_class_3)
pdf_class_1 = norm.pdf(z_class_1)


cov_matrix = probit_results.cov_params().values

grad_class_3 = np.array([1, 3, 1, 55]) * pdf_class_3
grad_class_1 = np.array([1, 1, 1, 55]) * pdf_class_1

var_delta_p = (grad_class_1- grad_class_3) @ cov_matrix @(grad_class_1- grad_class_3).T



In [57]:
std_error_delta_p = np.sqrt(var_delta_p)
std_error_delta_p

0.04078085646019446